# Iguazio Onboarding - Session 1

0. Platform Architecture + K8s Background

1. Iguazio Overview

2. Code to Microservice

3. More on MLRun Functions

## 0. Platform Architecture + Docker/K8s Background

![Docker](img/docker.png)
- Docker
    - A container is a standard unit of software that packages up code and all its dependencies so the application runs quickly and reliably from one computing environment to another.
    - A Docker container image is a lightweight, standalone, executable package of software that includes everything needed to run an application: code, runtime, system tools, system libraries and settings.

![K8s](img/k8s.png)
- Kubernetes (also known as K8s)
    - Open-source system for automating deployment, scaling, and management of containerized applications.
    - One stop shop - manages compute, networking, storage, permissions, scaling, etc.

- Platform Architecture
![Platform Architecture](img/platform_architecture.png)

## 1. Iguazio Overview
- UI Tour
- Project Management
- Managed Services

## 2. Code to Microservice
- Job Example (Python)
- Job Example (Notebook)
- Nuclio Example

- Package Python code into microservice to run on K8s

- ```python
fn = code_to_function(
    name="test",
    project="my_project",
    filename="test.py", # .py or .ipynb
    kind="job",
    image="mlrun/mlrun",
    handler="my_func"
)
fn.run()
```

- Add hooks for experiment tracking and more

- Experiment tracking
    ```python
    context.log_model()
    context.log_dataset()
    context.log_result()
    ```

- Pass inputs/parameters
    ```python
    context.get_param("batch_size")
    context.get_input("training_set")
    ```

- Logging
    ```python
    context.logger.info("Something happened")
    context.logger.error("Something went wrong")
    ```

### Job Example - Python

In [1]:
from mlrun import code_to_function

job = code_to_function(
    name="my_job",
    project="onboarding",
    filename="assets/job.py",
    kind="job",
    image="mlrun/mlrun",
    handler="my_func"
)
job.run(params={"value" : 123}, local=True)

> 2021-11-05 20:38:45,839 [info] starting run my-job-my_func uid=79c47d2193e64a90846323704946ef6f DB=http://mlrun-api:8080
> 2021-11-05 20:38:46,021 [info] Logging value
> 2021-11-05 20:38:46,032 [info] Logging CSV
> 2021-11-05 20:38:46,221 [info] Logging model


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
onboarding,...4946ef6f,0,Nov 05 20:38:45,completed,my-job-my_func,v3io_user=nickkind=owner=nickhost=jupyter-nick-77748f9fbf-pfrgl,,value=123,value=123,my_csvmy_model


> 2021-11-05 20:38:46,395 [info] run executed, status=completed


### Job Example - Notebook

In [2]:
from mlrun import auto_mount

job = code_to_function(
    name="my_notebook",
    project="onboarding",
    filename="assets/notebook.ipynb",
    kind="job",
    image="mlrun/mlrun",
    handler="main"
)
job.apply(auto_mount())
job.run(params={"test_size" : 0.3, "gamma" : 0.001, "C" : 100}, local=False)

> 2021-11-05 20:39:01,177 [info] starting run my-notebook-main uid=2809540325e440119ddd07c9b19bb214 DB=http://mlrun-api:8080
> 2021-11-05 20:39:01,366 [info] Job is running in the background, pod: my-notebook-main-gztk2
> 2021-11-05 20:39:06,596 [info] Getting data
> 2021-11-05 20:39:06,601 [info] Training model
> 2021-11-05 20:39:06,602 [info] Evaluating model
> 2021-11-05 20:39:06,605 [info] Logging datasets
> 2021-11-05 20:39:06,878 [info] Logging model
> 2021-11-05 20:39:07,021 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
onboarding,...b19bb214,0,Nov 05 20:39:06,completed,my-notebook-main,v3io_user=nickkind=jobowner=nickhost=my-notebook-main-gztk2,,test_size=0.3gamma=0.001C=100,accuracy=1.0,X_trainX_testy_trainy_testnotebook_model


> 2021-11-05 20:39:07,530 [info] run executed, status=completed


### Real-Time (Nuclio) Example

In [3]:
rt = code_to_function(
    name="my_rt",
    project="onboarding",
    filename="assets/rt.py",
    kind="nuclio",
    image="mlrun/mlrun",
    handler="handler"
)
rt.deploy()

> 2021-11-05 20:39:07,589 [info] Starting remote function deploy
2021-11-05 20:39:07  (info) Deploying function
2021-11-05 20:39:07  (info) Building
2021-11-05 20:39:07  (info) Staging files and preparing base images
2021-11-05 20:39:07  (info) Building processor image
2021-11-05 20:39:09  (info) Build complete
> 2021-11-05 20:39:15,350 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-onboarding-my-rt.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['default-tenant.app.us-sales-staging.iguazio-cd2.com:32463']}


'http://default-tenant.app.us-sales-staging.iguazio-cd2.com:32463'

In [4]:
%%time
rt.invoke(path="/")

> 2021-11-05 20:39:15,404 [info] invoking function: {'method': 'GET', 'path': 'http://nuclio-onboarding-my-rt.default-tenant.svc.cluster.local:8080/'}
CPU times: user 7.37 ms, sys: 1.13 ms, total: 8.49 ms
Wall time: 13.3 ms


{'result': '0.8249149472454543'}

## 3. More on MLRun Functions
- Basic configuration - Limits/requests
- Import/export
- Marketplace integration

### Basic Configuration - Limits/Requests

https://kubernetes.io/docs/concepts/configuration/manage-resources-containers/#requests-and-limits

In [13]:
job = code_to_function(
    name="my_notebook",
    project="onboarding",
    filename="assets/notebook.ipynb",
    kind="job",
    image="mlrun/mlrun",
    handler="main"
)
job.with_requests(cpu=2)
job.with_limits(cpu=2)
job.run(params={"test_size" : 0.3, "gamma" : 0.001, "C" : 100}, local=False)

> 2021-11-05 20:39:55,814 [info] starting run my-notebook-main uid=518d5368a83643c19f1c74f3ba423f98 DB=http://mlrun-api:8080
> 2021-11-05 20:39:55,980 [info] Job is running in the background, pod: my-notebook-main-t5mtj
> 2021-11-05 20:39:59,928 [info] Getting data
> 2021-11-05 20:39:59,932 [info] Training model
> 2021-11-05 20:39:59,934 [info] Evaluating model
> 2021-11-05 20:39:59,936 [info] Logging datasets
> 2021-11-05 20:40:00,450 [info] Logging model
> 2021-11-05 20:40:00,606 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
onboarding,...ba423f98,0,Nov 05 20:39:59,completed,my-notebook-main,v3io_user=nickkind=jobowner=nickhost=my-notebook-main-t5mtj,,test_size=0.3gamma=0.001C=100,accuracy=0.9555555555555556,X_trainX_testy_trainy_testnotebook_model


> 2021-11-05 20:40:02,148 [info] run executed, status=completed


### Export

In [14]:
# print(rt.to_yaml())

In [15]:
rt.export("assets/rt.yaml")

> 2021-11-05 20:40:16,732 [info] function spec saved to path: assets/rt.yaml


### Import (File)

In [16]:
from mlrun import import_function

In [17]:
new_rt = import_function("assets/rt.yaml")

In [18]:
# print(new_rt.to_yaml())

### Import (Marketplace)

In [19]:
# Reaches out to https://github.com/mlrun/functions (customizable)
fn = import_function("hub://describe")

In [20]:
# print(fn.to_yaml())